In [2]:
# opencv 3.4.2
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import os
import pandas as pd
from skimage.metrics import structural_similarity

cwd = os.getcwd()
queries_folder = cwd + "\\ug_data\Queries\\"
images_folder = cwd + "\\ug_data\Images\\"
results_folder = cwd + "\\Results\\"
#exampes_query_folder = cwd + "\\ug_data\examples\example_query\\"

if not os.path.exists(results_folder):
    os.makedirs(results_folder)

# return cropped image if bounding box annotation exists
def imreadBbox(imagePath,code=0):
    bbox_file_path = os.path.splitext(imagePath)[0] + ".txt"
    if os.path.exists(bbox_file_path):
        bbox_file = open(bbox_file_path, 'r')
        bbox = bbox_file.read().strip().split(' ')
        bbox_file.close()
        x = int(bbox[0])
        y = int(bbox[1])
        w = int(bbox[2])
        h = int(bbox[3])
        return cv.imread(imagePath,code)[y:y+h, x:x+w] #crop image if bbox file exists
    return cv.imread(imagePath,code)

def polylinesToBbox(dst):
    x1 = int(min([i[0][0] for i in dst]))
    y1 = int(min([i[0][1] for i in dst]))
    x2 = int(max([i[0][0] for i in dst]))
    y2 = int(max([i[0][1] for i in dst]))
    x1 = 0 if x1 < 0 else x1
    y1 = 0 if y1 < 0 else y1
    x2 = 0 if x2 < 0 else x2
    y2 = 0 if y2 < 0 else y2
    return x1,y1,x2,y2

In [ ]:
#orb matching
MIN_MATCH_COUNT = 10
orb = cv.ORB_create(nfeatures=3000)
bf = cv.BFMatcher(cv.NORM_HAMMING)

progress = 0
for filename1 in os.listdir(queries_folder):
    
    if os.path.splitext(filename1)[-1].lower() == '.jpg':
        
        img1 = imreadBbox(queries_folder+filename1)
        kp1, des1 = orb.detectAndCompute(img1,None)
        
        result_temp = ""
        for filename2 in os.listdir(images_folder):
            if os.path.splitext(filename2)[-1].lower() == '.jpg':
                img2 = cv.imread(images_folder+filename2,0)
                kp2, des2 = orb.detectAndCompute(img2,None)
                matches = bf.knnMatch(des1,des2, k=2)
                good = [m for m,n in matches if m.distance < 0.75*n.distance]
                
                h1,w1 = img1.shape[:2]
                h2,w2 = img2.shape[:2]
                x1,y1,x2,y2=0,0,0,0
                
                if len(good)>MIN_MATCH_COUNT:
                    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
                    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
                    M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
                    matchesMask = mask.ravel().tolist()
                    pts = np.float32([ [0,0],[0,h1-1],[w1-1,h1-1],[w1-1,0] ]).reshape(-1,1,2)
                    if M is not None:
                        dst = cv.perspectiveTransform(pts,M)
                        x1,y1,x2,y2 = polylinesToBbox(dst)
                
                # evaluate result
                x,y,w,h=x1,y1,x2-x1,y2-y1
                cond_1 = w > 0 and h > 0 and w < w2 and h < h2             # bbox exists and not too big
                cond_2 = x1 < w2 and x2 < w2 and y1 < h2 and y2 < h2       # bbox not out of range of shape
                cond_3 = w > 0.50*w1 and h > 0.50*h1 and w*h > 0.40*w1*h1  # bbox not too small 
                
                if cond_1 and cond_2 and cond_3:
                    img2_resized = cv.resize(img2[y:y+h, x:x+w], (w1,h1), interpolation = cv.INTER_AREA)
                    (score, diff) = structural_similarity(img1, img2_resized, win_size=101, full=True)
                    #edge1 = cv.Canny(img1,70,180)
                    #edge2 = cv.Canny(img2_resized,70,180)
                    #(score, diff) = structural_similarity(edge1, edge2, win_size=101, full=True)
                    result_temp += "%d\t%d\t%d\t%d\t%f\t%s" % (x1,y1,x2,y2,score,filename2) +"\n"
                else:
                    result_temp += "%d\t%d\t%d\t%d\t%f\t%s" % (x1,y1,x2,y2,-10,filename2) +"\n"
                 
                progress += 1
                if progress % 100 == 0:
                    print("prgress: %d/100000" % (progress))
                    
        filePath = results_folder+ "3_orb_q"+ os.path.splitext(filename1)[0] +".txt"
        f = open(filePath, "w")
        f.write(result_temp)
        f.close()

In [ ]:
# Generate Top 10 Result
resultList = pd.DataFrame()
for n in range(1,21):
    df = pd.read_csv(results_folder+'3_orb_q{:02d}.txt'.format(n), delimiter = "\t", names=['x1','y1','x2','y2','ssim','filename'])
    df = df.sort_values(by=['ssim'],ascending=False)
    resultList = resultList.append(df.iloc[0:10])

fig=plt.figure(figsize=(255, 255))
columns, rows = 10, 20
for i in range(1, columns*rows +1):
    annotation = resultList.iloc[i-1]
    filename = images_folder+str(annotation['filename'])
    img = cv.cvtColor(cv.imread(filename), cv.COLOR_BGR2RGB)
    x1,y1,x2,y2 = int(annotation['x1']),int(annotation['y1']),int(annotation['x2']),int(annotation['y2'])
    cv.rectangle(img, (x1,y1), (x2,y2), (255,0), 5)
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
plt.savefig(results_folder+"3_top10_results.png")
plt.show()

In [4]:
# Generate Rank List
temp = ""
for n in range(1,11):
    temp += "Q%d: " % (n)
    df = pd.read_csv(results_folder+'3_orb_q{:02d}.txt'.format(n), delimiter = "\t", names=['x1','y1','x2','y2','ssim','filename'])
    df = df.sort_values(by=['ssim'],ascending=False)
    arr = df.loc[:,['filename']].values.tolist()
    arr = [os.path.splitext(a[0])[0].lstrip("0") for a in arr]
    temp += " ".join(arr) +"\n"

filePath = results_folder+ "3_rankList.txt"
f = open(filePath, "w")
f.write(temp)
f.close()

In [3]:
# Calculate MAP
from __future__ import division

groundtruthList = cwd + "\\ug_data\examples\example_result\\rank_groundtruth.txt"
rankList = results_folder+ "3_rankList.txt"

# average precision
def ave_pre(array):
    ap = 0
    for idx, m in enumerate(array):
        ap_m = (idx + 1) / (m + 1)
        ap = ap + ap_m
    ap = ap / len(array)
    return ap

with open(groundtruthList) as f:
    rank_line = f.read().splitlines()
# read retrieved num of the relevant images
with open(rankList) as f:
    rank_result = f.read().splitlines()

# compute mean average precision
ap_sum = 0
for idx, line in enumerate(rank_result):
    line_str = line.split()
    query_num = int(line_str[0][1]) - 1
    result_num = [int(x) for x in line_str[1:]]
    rank_str = rank_line[idx].split()
    rank_gt = [int(x) for x in rank_str[1:]]
    find_idx = []
    for num in rank_gt:
        ind = np.where(np.array(result_num) == num)
        find_idx.extend(ind)
    find_idx = np.array(find_idx).reshape(len(find_idx),)
    find_idx = np.sort(find_idx)
    ap = ave_pre(find_idx)
    print ("Average Precision of Q%d: %.4f"%(idx+1, ap))
    ap_sum = ap_sum + ap
print ("Mean Average Precision: %f" %(ap_sum / len(rank_result)))

Average Precision of Q1: 0.1470
Average Precision of Q2: 0.4602
Average Precision of Q3: 0.7421
Average Precision of Q4: 0.0049
Average Precision of Q5: 0.0444
Average Precision of Q6: 0.2553
Average Precision of Q7: 0.0055
Average Precision of Q8: 0.0331
Average Precision of Q9: 0.0058
Average Precision of Q10: 0.1072
Mean Average Precision: 0.180546
